In [ ]:
!pip install -U -q transformers accelerate bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from huggingface_hub import login
import torch
import os
import json
import ast
import re
from tqdm import tqdm
# from google.colab import drive #Use it if you're going to use a colab env
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# hugging face token
token = input()
login(token=token)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)
model = torch.compile(model)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=False,
    num_beams=1,
    return_full_text=False,
)

In [ ]:
def adjust_json(json):
  try:
      # try to direct interprest received text
      if isinstance(json, dict):
          return json

      # Fixing common found error
      # changing simple qutoes to double quotes
      json_corrigido = re.sub(r"(?<!\\)'", '"', json)

      try:
          return json.loads(json_corrigido)
      except json.JSONDecodeError:
          pass

      # Try a fallback function
      return ast.literal_eval(json)

  except Exception as e:
      print("Error on reading json file:", e)
      return None

In [ ]:
description_extractive = "Um resumo extrativo extrai somente sentenças literais não alteradas das avaliações originais que melhor representem o sentimento geral e os principais pontos. O resumo extrativo não faz uso de palavras ou sentenças não presentes nas avaliações originais, como adições e conectivos não existentes."

In [ ]:
description_abstractive = "Um resumo abstrativo cria uma nova sentença (ou conjunto de sentenças) que sintetiza o sentimento geral e os principais pontos, podendo utilizar novas palavras e reestruturações linguísticas, sem se limitar ao vocabulário presente nas avaliações originais."

In [ ]:
description_extractive = "Um resumo extrativo extrai somente sentenças literais não alteradas das avaliações originais que melhor representem o sentimento geral e os principais pontos. O resumo extrativo não faz uso de palavras ou sentenças não presentes nas avaliações originais, como adições e conectivos não existentes."
description_abstractive = "Um resumo abstrativo cria uma nova sentença (ou conjunto de sentenças) que sintetiza o sentimento geral e os principais pontos, podendo utilizar novas palavras e reestruturações linguísticas, sem se limitar ao vocabulário presente nas avaliações originais."
prompt = """
  Considere as avaliações de produto abaixo. Gere um {category}:

  {description}

  Sua resposta deve conter o {category}, claramente separado e identificado num formato json válido.
  Não use do seu conhecimento prévio sobre o assunto para gerar o resumo, portanto considere somente as avaliações enviadas.
  O resumo deve conter entre 2 e 4 sentenças.
  Formato json válido:
  {{
    "assunto": "{subject}",
    "resumo": "seu resumo aqui"
  }}
  Não inclua saudações, explicações ou qualquer outro texto dentro ou fora do json.

  Avaliações:
  {evaluations}
  """

categories = ["resumo extrativo", "resumo abstrativo"]
descriptions = [description_extractive, description_abstractive]
DATA_PATH = "OpiSums-PT/Textos_Fontes/" #folder generated using utils/ud_tagger.py function
subjects = []
for file in os.listdir(DATA_PATH):
  if os.path.isdir(DATA_PATH + file) and not file.endswith(".ipynb_checkpoints"):
    subjects.append(file)
print(subjects)

['Capitaes-da-Areia', '1984', 'LG-Smart-TV', 'Fala-Serio-Pai', 'Galaxy-SIII', 'Iphone-5', 'Crepusculo', 'Fala-Serio-Amiga', 'Fala-Serio-Amor', 'Fala-Serio-Professor', 'Fala-Serio-Mae', 'Ensaio-Sobre-a-Cegueira', 'Samsung-Smart-TV', 'O-Reverso-da-Medalha', 'O-Apanhador-no-Campo-de-Centeio', 'O-Outro-Lado-da-Meia-Noite', 'Se-Houver-Amanha']


In [ ]:
def clean_extractive_human_text(text):
    # Remove tags like <...> at the end of each line
    lines = text.splitlines()
    cleaned_lines = [re.sub(r'\s*<[^>]+>\s*$', '', linha) for linha in lines]
    return '\n'.join(cleaned_lines)

In [ ]:
OUTPUT_PATH = "resumos_gerados/"

for subject in subjects:
  evaluations = []
  evaluations_path = DATA_PATH + subject + "/"
  for file in os.listdir(evaluations_path):
    if file.endswith(".txt"):
      with open(evaluations_path + file, "r") as f:
        eval = f.read()
        evaluations.append(clean_extractive_human_text(eval))

  # Use answer for each pair categorie + description
  for category, description in zip(categories, descriptions):
    print(f"\Generation for Category: {category}, Subject: {subject}\n")

    formatted_prompt = prompt.format(category=category, subject=subject.strip(),description=description.strip(), evaluations="\n".join(evaluations))
    result_json = ''
    result = generator(formatted_prompt)

    try:
      result_json = json.loads(result[0]['generated_text'])
    except json.JSONDecodeError:
      print("Error found")
      result_json = adjust_json(result[0]['generated_text'])
      print(result_json)


    if not os.path.exists(OUTPUT_PATH + subject):
      os.makedirs(OUTPUT_PATH + subject)
    if not os.path.exists(OUTPUT_PATH + subject + "/llm"):
      os.makedirs(OUTPUT_PATH + subject + "/llm")

    resume_file = f"resumo_{category.split(' ')[1]}.txt"
    resume_path = OUTPUT_PATH + subject + "/llm/" + resume_file

    print("File saved in:", resume_path)
    with open(resume_path, "w") as f:
      if result_json:
        f.write(result_json["resumo"])
      else:
        f.write(f"Bad generation.\nLLM RESPONSE:{result[0]['generated_text']}")